<a href="https://colab.research.google.com/github/aswani-sh/AICTE_Internship_june2025/blob/main/Garbageclassificationweek2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Layer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Rescaling, GlobalAveragePooling2D
from tensorflow.keras import layers, optimizers, callbacks
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.applications import EfficientNetV2B2
from sklearn.metrics import confusion_matrix, classification_report
import gradio as gr
import os


# --- For Google Drive ---
from google.colab import drive
drive.mount('/content/drive')
dataset_dir = "/content/garbage_dataset"
  # 🔁 Replace this with your actual path

import zipfile

# Unzip the dataset
zip_path = "/content/drive/MyDrive/garbage (1).zip"  #path
extract_path =  "/content/garbage_dataset/TrashType_Image_Dataset"
     # Folder where ZIP contents will be extracted

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

dataset_dir = extract_path  # Point to the extracted folder, not the zip file


# Dataset parameters
image_size = (124, 124)
batch_size = 32
seed = 42

# Load datasets
train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="training",
    seed=seed,
    shuffle=True,
    image_size=image_size,
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="validation",
    seed=seed,
    shuffle=True,
    image_size=image_size,
    batch_size=batch_size
)




val_class = val_ds.class_names
print("Classes:", val_class)




import os

for root, dirs, files in os.walk(dataset_dir):
    level = root.replace(dataset_dir, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for f in files[:5]:  # show first 5 files only
        print(f"{subindent}{f}")


import os

def find_dataset_root(base_path):
    for root, dirs, files in os.walk(base_path):
        # true root: no image files directly here but multiple subfolders
        if len(dirs) >= 2 and all(os.path.isdir(os.path.join(root, d)) for d in dirs) and not any(f.lower().endswith(('.jpg','.png')) for f in files):
            return root
    return None

dataset_dir = find_dataset_root("/content/garbage_dataset")
print("✅ Detected dataset directory:", dataset_dir)  # Should print path ending with .../TrashType_Image_Dataset






# Get the total number of batches in the validation dataset
val_batches = tf.data.experimental.cardinality(val_ds)

# Split the validation dataset into two equal parts:
# First half becomes the test dataset
test_ds = val_ds.take(val_batches // 2)

# Second half remains as the validation dataset
val_dat = val_ds.skip(val_batches // 2)

# Optimize test dataset by caching and prefetching to improve performance
test_ds_eval = test_ds.cache().prefetch(tf.data.AUTOTUNE)


print(train_ds.class_names)
print(val_class)
print(len(train_ds.class_names))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 18 files belonging to 1 classes.
Using 15 files for training.
Found 18 files belonging to 1 classes.
Using 3 files for validation.
Classes: ['garbage']
TrashType_Image_Dataset/
  garbage/
    TrashType_Image_Dataset/
      cardboard/
      glass/
      plastic/
        plastic_017.jpg
        plastic_005.jpg
        plastic_012.jpg
        plastic_016.jpg
        plastic_002.jpg
      trash/
      metal/
      paper/
✅ Detected dataset directory: /content/garbage_dataset
['garbage']
['garbage']
1
